In [27]:
import os
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import csv
import seaborn as sns
from skimage.feature import hog
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from joblib import Parallel, delayed
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay)
import time

In [9]:
class_names = {
    0: "Speed limit (20km/h)", 1: "Speed limit (30km/h)", 2: "Speed limit (50km/h)",
    3: "Speed limit (60km/h)", 4: "Speed limit (70km/h)", 5: "Speed limit (80km/h)",
    6: "End of speed limit (80km/h)", 7: "Speed limit (100km/h)", 8: "Speed limit (120km/h)",
    9: "No passing", 10: "No passing for vehicles over 3.5 metric tons",
    11: "Right-of-way at the next intersection", 12: "Priority road", 13: "Yield",
    14: "Stop", 15: "No vehicles", 16: "Vehicles over 3.5 metric tons prohibited",
    17: "No entry", 18: "General caution", 19: "Dangerous curve to the left",
    20: "Dangerous curve to the right", 21: "Double curve", 22: "Bumpy road",
    23: "Slippery road", 24: "Road narrows on the right", 25: "Road work",
    26: "Traffic signals", 27: "Pedestrians", 28: "Children crossing",
    29: "Bicycles crossing", 30: "Beware of ice/snow", 31: "Wild animals crossing",
    32: "End of all speed and passing limits", 33: "Turn right ahead",
    34: "Turn left ahead", 35: "Ahead only", 36: "Go straight or right",
    37: "Go straight or left", 38: "Keep right", 39: "Keep left",
    40: "Roundabout mandatory", 41: "End of no passing",
    42: "End of no passing for vehicles over 3.5 metric tons"
}

In [5]:
augmented_I_RGB = np.load('augmented_I_RGB.npy')      # Shape: (N, 32, 32, 3)
augmented_I_Gray = np.load('augmented_I_Gray.npy')    # Shape: (N, 32, 32)
augmented_L = np.load('augmented_L.npy')              # Shape: (N,)

#Verify shapes
print(f"RGB images: {augmented_I_RGB.shape}")
print(f"Grayscale images: {augmented_I_Gray.shape}")
print(f"Labels: {augmented_L.shape}")

RGB images: (78418, 32, 32, 3)
Grayscale images: (78418, 32, 32)
Labels: (78418,)


In [21]:
def extract_hog_features(image):
    return hog(image, pixels_per_cell=(4, 4), cells_per_block=(2, 2), visualize=False)

X = np.array(Parallel(n_jobs=-1)(delayed(extract_hog_features)(img) for img in augmented_I_Gray))
y = augmented_L  

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42,)

rf = RandomForestClassifier(n_estimators=400, random_state=42,n_jobs=-1)
start_time = time.time()
rf.fit(X_train, y_train)
training_time = time.time() - start_time
y_pred = rf.predict(X_test)


#Evals
accuracy = accuracy_score(y_test, y_pred)
print(f"\n=== Results ===")
print(f"Training time: {training_time/60:.1f} minutes")
print(f"Training Accuracy: {rf.score(X_train, y_train):.4f}")
print(f"Test Accuracy: {rf.score(X_test, y_test):.4f}")
print("Classification Report:\n", classification_report(y_test, y_pred, target_names=[class_names[i] for i in sorted(class_names.keys())]))


=== Results ===
Training time: 1.3 minutes
Training Accuracy: 1.0000
Test Accuracy: 0.9638
Classification Report:
                                                      precision    recall  f1-score   support

                               Speed limit (20km/h)       1.00      0.73      0.84        67
                               Speed limit (30km/h)       0.94      0.91      0.92       872
                               Speed limit (50km/h)       0.98      0.99      0.99       830
                               Speed limit (60km/h)       0.92      0.98      0.95       572
                               Speed limit (70km/h)       1.00      1.00      1.00       825
                               Speed limit (80km/h)       1.00      1.00      1.00       890
                        End of speed limit (80km/h)       1.00      0.97      0.98       320
                              Speed limit (100km/h)       0.99      0.98      0.99       270
                              Speed limit (120

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42,stratify=y)

rf = RandomForestClassifier(n_estimators=400, random_state=42,n_jobs=-1)
start_time = time.time()
rf.fit(X_train, y_train)
training_time = time.time() - start_time
y_pred = rf.predict(X_test)

#Evals
accuracy = accuracy_score(y_test, y_pred)
print(f"\n=== Results ===")
print(f"Training time: {training_time/60:.1f} minutes")
print(f"Training Accuracy: {rf.score(X_train, y_train):.4f}")
print(f"Test Accuracy: {rf.score(X_test, y_test):.4f}")
print("Classification Report:\n", classification_report(y_test, y_pred, target_names=[class_names[i] for i in sorted(class_names.keys())]))


=== Results ===
Training time: 1.4 minutes
Training Accuracy: 1.0000
Test Accuracy: 0.9666
Classification Report:
                                                      precision    recall  f1-score   support

                               Speed limit (20km/h)       0.98      0.69      0.81        84
                               Speed limit (30km/h)       0.95      0.92      0.94       888
                               Speed limit (50km/h)       0.96      1.00      0.98       804
                               Speed limit (60km/h)       0.94      0.98      0.96       528
                               Speed limit (70km/h)       1.00      1.00      1.00       840
                               Speed limit (80km/h)       0.99      1.00      1.00       864
                        End of speed limit (80km/h)       1.00      0.97      0.99       312
                              Speed limit (100km/h)       0.99      0.99      0.99       252
                              Speed limit (120